# Convert a species_properties file to json

TODO: add to fair-calibrate

This is so we can bang it into openscm-runner and recycle the default parameters for fair as a set of flat configs

We need a 1D flat config set as we will create N copies of it in a list to combine with the configs that do vary - these will be a list that correspond to species_configs in the calibrated constrained parameters CSV

we want to remove the params that do vary, otherwise we have two keys that are the same, and this is bad

others: 11 params are energy balance model, co2 initial is initial conditions, seed can be calculated from config # but may be unnecessary if we only run deterministic

TODO: any initial conditions that will vary by run also need to be excluded, for example if we do offline spinup runs and initialise all gas boxes in say 2019.

cheers to https://medium.com/@techwithjulles/python-a-program-that-converts-a-csv-file-to-a-json-file-80c18446ac0b

In [1]:
import csv
import json

import numpy as np

In [2]:
with open('species_configs_properties_calibration1.2.0.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row for row in reader]

In [3]:
non_forcing_scale_params_that_vary = (
    'CO2|iirf_0',
    'CO2|iirf_airborne',
    'CO2|iirf_uptake',
    'CO2|iirf_temperature',
    'BC|erfari_radiative_efficiency',
    'OC|erfari_radiative_efficiency',
    'Sulfur|erfari_radiative_efficiency',
    'NOx|erfari_radiative_efficiency',
    'VOC|erfari_radiative_efficiency',
    'NH3|erfari_radiative_efficiency',
    'CH4|erfari_radiative_efficiency',
    'N2O|erfari_radiative_efficiency',
    'Equivalent effective stratospheric chlorine|erfari_radiative_efficiency',
    'Sulfur|aci_shape',
    'BC|aci_shape',
    'OC|aci_shape',
    'Aerosol-cloud interactions|aci_scale',
    'CH4|ozone_radiative_efficiency',
    'N2O|ozone_radiative_efficiency',
    'Equivalent effective stratospheric chlorine|ozone_radiative_efficiency',
    'CO|ozone_radiative_efficiency',
    'VOC|ozone_radiative_efficiency',
    'NOx|ozone_radiative_efficiency'
)

In [4]:
out_data = {}

for item in data:
    for subitem in item:
        if subitem == 'forcing_scale':  # this is always overridden
            continue  
        value = item[subitem]
        try:
            value=int(value)
        except ValueError:
            try:
                value=float(value)
                if np.isnan(value):  # nan will be default, so don't carry 'em round
                    continue
            except ValueError:
                pass
        if f'{item["name"]}|{subitem}' in non_forcing_scale_params_that_vary:  # don't include in defaults if it's something we vary
            continue
            
        out_data[f'{item["name"]}|{subitem}'] = value

In [5]:
with open('default_configs_slim.json', 'w') as jsonfile:
    json.dump(out_data, jsonfile)